# Merge Clean Scripts with Movie Information

In [38]:
#Import libaries 
import pandas as pd
import re
import numpy as np 
import os

In [39]:
os.getcwd()
os.chdir('/Users/Sofie/Disney Thesis')

In [40]:
# import clean scripts
scripts = pd.read_pickle("Clean_Scripts_03mar.pickle")

FileNotFoundError: [Errno 2] No such file or directory: 'Clean_Scripts_03mar.pickle'

In [ ]:
# If a pages is collected through a PDF or other scripts['html']  is Nan --> change to other 
site = []
for movie in scripts['Site']:
    if pd.isna(movie) == True:
        page = "Other"
    else:
        page = movie
    site.append(page)

scripts['Site'] = site
    

In [ ]:
# import movie info
movie_info = pd.read_excel('Disney info movies.xlsx')


In [ ]:
# merge scripts with movie information collected from kaggle 
movies = pd.merge(scripts, movie_info, on='Movie', how='outer')
movies = movies.sort_values(by='Year', ascending=True)
movies = movies.reset_index(drop=True)

movies.to_pickle("all movie information.pickle")

In [41]:
movies

,Movie,Script,Site,html,Clean_Script,Running time minutes (int),Box office (float),Starring,Box office,imdb_id,averageRating,numVotes,Year,Disney or pixar,Budget (millions),Writers,Writers_male,Writers_female
0,Snow White and the Seven Dwarfs,"<li>Narrator: Once upon a time, there lived a ...",fandom,p_split,Character ...,83,418000000,"['Adriana Caselotti', 'Lucille La Verne', 'Har...","$ 418.000.000,00",tt0029583,7.6,192877,1937,Disney,1.49,"Ted Sears (story adaptation), Richard Creedon...",1,1
1,Pinocchio,Jiminy Cricket:</b> (<i>singing</i>) ♪ <i>When...,fandom,b-tag,...,88,164000000,"['Cliff Edwards', 'Dickie Jones', 'Christian R...","$ 164.000.000,00",tt0032910,7.4,139865,1940,Disney,2.60,"Carlo Collodi (from the story by), Ted Sears (...",1,0
2,Fantasia,NaN,NaN,NaN,NaN,126,83300000,"['Leopold Stokowski', 'Deems Taylor']","$ 83.300.000,00",tt0032455,7.7,93503,1940,Disney,2.28,"Joe Grant (story direction), Dick Huemer (stor...",1,1
3,Bambi,NaN,Other,NaN,Character ...,70,267400000,see below,"$ 267.400.000,00",t0036326,7.3,138736,1942,Disney,0.86,"Felix Salten(from the story by), Perce Pearce(...",1,0
4,Saludos Amigos,NaN,NaN,NaN,NaN,42,1135000,"['Lee Blair', 'Mary Blair', 'Pinto Colvig', 'W...","$ 1.135.000,00",tt0034492,7.3,138659,1942,Disney,NaN,"Ted Sears (story research), William Cottrell (...",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Soul,NaN,NaN,NaN,NaN,100,120957731,NaN,"$ 120.957.731,00",tt2948372,8.1,298469,2020,Pixar,150.00,"Pete Docter (story by, )Mike Jones (story by),...",1,0
79,Onwards,NaN,Other,NaN,Character ...,102,141940000,NaN,"$ 141.940.000,00",tt7146812,7.4,135518,2020,Pixar,200.00,"Dan Scanlon (original story by, )Keith Bunin (...",1,0
80,Encanto,NaN,Other,NaN,Character ...,102,233912000,NaN,"$ 233.912.000,00",tt2953050,7.3,123246,2021,Disney,50.00,"Charise Castro Smith(screenplay by), Jared Bus...",1,1
81,Luca,NaN,Other,NaN,Character ...,95,49750471,NaN,"$ 49.750.471,00",tt12801262,7.5,136257,2021,Pixar,175.00,"Enrico Casarosa (story by), Jesse Andrews (sto...",1,0


# Clean non-values from clean scripts

In [42]:
# subdata - dataset with scripts
movies_script = movies[movies["Clean_Script"].notna()]
movies_script = movies_script.reset_index(drop = True)


In [43]:
#drop rows if character is nan

for movie in movies_script['Clean_Script']:
    for index, row in movie.iterrows():
        if pd.isna(row['Character']) == True:
            movie.drop(index, inplace=True)
        elif row['Line'] == "":
            movie.drop(index, inplace=True)
        elif len(row['Character']) > 35: #kan bruges til at printe cleanet forkerte sætninger
            movie.drop(index, inplace=True)
        

# Gender of Characters

In [44]:
# load names 
df_male = pd.read_excel("Male_names.xlsx")
df_female = pd.read_excel('Female_names.xlsx')

In [45]:
# lower names
df_male["Name"] = df_male["Name"].str.lower()
df_female["Name"] = df_female["Name"].str.lower()

In [46]:
# functions for removing extra white spaces

def remove_first_end_spaces(string):
    return "".join(string.rstrip().lstrip())

# Remove all extra spaces
def remove_all_extra_spaces(string):
    return " ".join(string.split())


In [47]:
# all characters name --> lower 
names = []
for index, row in movies_script.iterrows():
    lower_names = []
    for name in row['Clean_Script']['Character']:
        name = str(name)
        name = name.lower()
        name = re.sub("v/o|v/0", "", name)
        name = remove_first_end_spaces(name)
        name = remove_all_extra_spaces(name)
        
        lower_names.append(name)
    names.append(lower_names)
    
for i in range(len(movies_script)):
    movies_script['Clean_Script'][i]['Character_lower'] = names[i]


In [48]:
men = ['men', 'man', 'duke', 'king', 'prince ', 'boy', 'father', 'dad', 
       'pap','mr', 'male', 'brother', 'guard', 'jack', 'timothy', 'jiminy','geppetto']
women = ['girl', 'women', 'woman', 'princess', 'queen', 'sister', 'lady', 'waitress', 'barbie',
         'mother', 'mum', 'mrs', 'madame', 'female', 'mam', 'abuelita', 'abuela', 'mom']

In [49]:
# Are the Characters female or male? 
movies_gender = []
for index, row in movies_script.iterrows():
    gender_list = []
    if isinstance(row['Clean_Script'], pd.DataFrame):
        # if part of character name is in one of the list --> either male or female
        for character in row['Clean_Script']['Character_lower']:
            if any(word in character for word in list(df_female['Name'])):
                gender = "Female"
            elif any(word in character for word in list(df_male['Name'])):
                gender = "Male"
            
            
            elif any(word in character for word in women): 
                gender = "Female"
            elif any(word in character for word in men): 
                gender = "Male"
            else:
                gender = np.nan
            gender_list.append(gender)

    
    else:
        gender_list = np.nan

    movies_gender.append(gender_list)
    
for i in range(len(movies_script)):
    movies_script['Clean_Script'][i]['Gender'] = movies_gender[i]

In [50]:
movies_script = movies_script.reset_index()

In [51]:
# print unikkelister af characters uden female/male 

names_wo_gender = []
movie_name = []
i = -1
for movie in movies_script['Clean_Script']:
    i = i+1
    
    for index, row in movie.iterrows():
        if pd.isna(row['Gender']) == True:
            names_wo_gender.append(row['Character_lower'])
            movie_name.append(i)

names_gender = pd.DataFrame()
names_gender['Movie'] = movie_name
names_gender['Character_name'] = names_wo_gender

names_gender = names_gender.drop_duplicates()
names_gender = names_gender.reset_index(drop = True)

names_gender = names_gender.rename(columns={"Movie": "index"})
names_gender = names_gender.merge(movies_script[['index', 'Movie']])
names_gender

names_gender.to_excel('Unique_names_new.xlsx')

In [52]:
len(names_gender)

1060

In [53]:
#names_gender.groupby('Movie').count()[40:]

In [54]:
# open gender of characters after manual validation
validated_names = pd.read_excel('Unique_names_validated.xlsx')
validated_names["Gender"].replace({"F": "Female", "M": "Male", "G": "Group", "N": "Non-human"}, inplace=True)
validated_names = validated_names.rename(columns={"Character_name": "Character_lower"})
validated_names

,Unnamed: 0,index,Character_lower,Movie,Gender
0,0,0,narrator,Snow White and the Seven Dwarfs,Male
1,1,0,grimhilde,Snow White and the Seven Dwarfs,Female
2,2,0,magic,Snow White and the Seven Dwarfs,Non-human
3,3,0,snow,Snow White and the Seven Dwarfs,Female
4,4,0,echo,Snow White and the Seven Dwarfs,Non-human
...,...,...,...,...,...
1209,1217,62,namaari,Raya and the Last Dragon,Female
1210,1218,62,boun,Raya and the Last Dragon,Male
1211,1219,62,fang officer,Raya and the Last Dragon,Female
1212,1220,62,noi,Raya and the Last Dragon,Female


In [55]:
#pd.to_pickle(validated_names, "validated_names.pickle")
movies_script.to_pickle("movies_script.pickle")
validated_names.to_pickle("validate_names.pickle")

In [56]:
# Look up gender in validated_names 


#convert validated_names to dict
validated_names_dict = (
    validated_names.groupby('Movie')
                   .apply(lambda movie: dict(zip(movie['Character_lower'], 
                                                 movie['Gender'])))
                   .to_dict())

# Look up 
gender_in_movies = []
movies_script2 = movies_script.copy()
for _, r0 in movies_script2.iterrows():
    gender_list = []
    movie = r0["Movie"]
    for _, r1 in r0["Clean_Script"].iterrows():
        character    = r1["Character_lower"]
        if character in validated_names_dict[movie]:
            gender = validated_names_dict[movie][character]
        else:
            gender = r1["Gender"]
        gender_list.append(gender)
    gender_in_movies.append(gender_list)

#Append new gender to clean_scripts     
for i in range(len(movies_script)):
    movies_script['Clean_Script'][i]['Gender_new'] = gender_in_movies[i]

In [57]:
# delete all lines with no gender   
for movie in movies_script['Clean_Script']:
    for index, row in movie.iterrows():
        if pd.isna(row['Gender_new']) == True:
            movie.drop(index, inplace=True)

In [58]:
#save to pickle
movies_script.to_pickle("final_gendered_scripts.pickle")

In [59]:
#Print list with all unique characters including movies and gender

names_wo_gender = []
gender_list = []
movie_name = []
i = -1
for movie in movies_script['Clean_Script']:
    i = i+1
    
    for index, row in movie.iterrows():
        names_wo_gender.append(row['Character'])
        gender_list.append(row['Gender_new'])
        movie_name.append(i)

validate_gender = pd.DataFrame()
validate_gender['Movie'] = movie_name
validate_gender['Character_name'] = names_wo_gender
validate_gender['Gender'] = gender_list

validate_gender = validate_gender.drop_duplicates()
validate_gender = validate_gender.reset_index(drop = True)

validate_gender = validate_gender.rename(columns={"Movie": "index"})
validate_gender = validate_gender.merge(movies_script[['index', 'Movie']])
validate_gender

validate_gender.to_excel('all_names_gender.xlsx')

In [60]:
len(validate_gender)

2560

In [61]:
# replace gender if gender is validated to be wrong
updated_names = pd.read_excel('wrong_names_validated.xlsx')
errors = updated_names[updated_names["Error"] == 'FEJL']
val_names = updated_names[updated_names["Gender_New"].notna()]

In [62]:
# convert the list of errors and manually validated gender of names 

#convert validated_names to dict
val_names_dict = (
    val_names.groupby('Movie')
                   .apply(lambda movie: dict(zip(movie['Character'], 
                                                 movie['Gender_New'])))
                   .to_dict())

# convert errors to dict 
error_dict = (
    errors.groupby('Movie')
                   .apply(lambda movie: dict(zip(movie['Character'], 
                                                 movie['Error'])))
                   .to_dict())

In [63]:
# Replace gender if validated to be another gender 

gender_validated = []
movies_script2 = movies_script.copy()
for _, r0 in movies_script2.iterrows():
    gender_list = []
    movie = r0["Movie"]
    if movie in val_names_dict:
        
        for _, r1 in r0["Clean_Script"].iterrows():
            character    = r1["Character"]
            if character in val_names_dict[movie]:
                gender = val_names_dict[movie][character]
            else:
                gender = r1["Gender_new"]
            gender_list.append(gender)
        gender_validated.append(gender_list)
    else:
        for _, r1 in r0["Clean_Script"].iterrows():
            gender = r1["Gender_new"]
            gender_list.append(gender)
        gender_validated.append(gender_list)
        

#Append new gender to clean_scripts     
for i in range(len(movies_script)):
    movies_script['Clean_Script'][i]['Gender_updated'] = gender_validated[i]

In [64]:
# if in error dict --> remove 
error_validated = []


for _, r0 in movies_script.iterrows():
    gender_list = []
    movie = r0["Movie"]
    if movie in error_dict:

        for index, r1 in r0["Clean_Script"].iterrows():
            character    = r1["Character"]
            #character = remove_all_extra_spaces(character)
            if character in error_dict[movie]:
                #print(r1['Character'])
                r0["Clean_Script"].drop(index, inplace=True)     


In [65]:
# remove last space 

character_removespace = []

for index, row in movies_script.iterrows():
    character_updated = []
    if isinstance(row['Clean_Script'], pd.DataFrame):
        # if part of character name is in one of the list --> either male or female
        for character in row['Clean_Script']['Character']:
            character = remove_all_extra_spaces(character)
            character = re.sub(r':', '', character)
            character_updated.append(character)
    else:
        character_updated = np.nan

    character_removespace.append(character_updated)
    
line_removespace = []
for index, row in movies_script.iterrows():
    line_updated = []
    if isinstance(row['Clean_Script'], pd.DataFrame):

        for line in row['Clean_Script']['Line']:
            line = remove_all_extra_spaces(line)
            line_updated.append(line)
    else:
        line_updated = np.nan

    line_removespace.append(line_updated)
    
for i in range(len(movies_script)):
    movies_script['Clean_Script'][i]['Character'] = character_removespace[i]
    movies_script['Clean_Script'][i]['Line'] = line_removespace[i]

In [66]:
# test scripts 
os.getcwd()
os.chdir('/Users/Sofie/Disney Thesis/Test_Scripts')

for index, row in movies_script.iterrows():
    movie = row['Movie']
    name = movie + ".xlsx"
    script = row['Clean_Script']
    script.to_excel(name)

In [67]:
# save data frame 
movies_script.to_pickle("scripts_clean_final.pickle")

# Minor cleaning 

We found a couple of mistakes in the gender assignment, when we started plotting the descriptive analysis. These are implemented here

In [68]:
scripts = pd.read_pickle("scripts_clean_final.pickle")


In [69]:
scripts['Clean_Script'][3]
gender = []
for index, row in scripts['Clean_Script'][3].iterrows():
    
    if row['Character_lower'] == "joe":
        gender.append("Male")
    else:
        gender.append(row['Gender_updated'])

scripts['Clean_Script'][3]['Gender_updated'] = gender

In [70]:
lines = []
for line in scripts['Clean_Script'][27]['Line']:
    line = line.split(" - ")[0]
    lines.append(line)
scripts['Clean_Script'][27]['Line'] = lines

In [71]:
# save data frame 
scripts.to_pickle("scripts_clean_final.pickle")